### predicting if loans will be paid back
clean data: filter and deal with missing values <br>
try different prediction methods

In [33]:
import pandas as pd
import numpy 


loans_2007=pd.read_csv('loans_2007.csv')
#print(loans_2007.head(1),loans_2007.shape[1])

#remove columns that give away information about the future

loans_2007.drop(['id','member_id','funded_amnt','funded_amnt_inv','grade','sub_grade','emp_title','issue_d','pymnt_plan'],axis=1, inplace=True)

loans_2007=loans_2007.drop(['zip_code','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp'],axis=1)

loans_2007=loans_2007.drop(['total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_amnt','last_pymnt_d'],axis=1)

#Removing single value columns ##

columns_to_drop=[]
for col in loans_2007.columns:
    #loans_2007[col]=loans_2007[col].dropna()
    if len(loans_2007[col].dropna().unique())==1:
        columns_to_drop.append(col)
loans_2007=loans_2007.drop(columns_to_drop,axis=1)

#print(loans_2007.head(1),loans_2007.shape[1])


#decide on the target column: loan_status
#print(loans_2007['loan_status'].value_counts())

#goal: binary classification if loan will be paid back or not, reduction to two outcomes
loans_2007=loans_2007.loc[(loans_2007['loan_status']=='Fully Paid')|(loans_2007['loan_status']=='Charged Off')]
map_dict={'Fully Paid':1,
          'Charged Off':0}

loans_2007['loan_status']=loans_2007['loan_status'].replace(map_dict)
print('classifiers of loan_status: '+str(loans_2007['loan_status'].unique()))
print('amount of columns left: '+str(loans_2007.shape[1]))
loans_2007.head(1)

#result is filtered data
                          


        

classifiers of loan_status: [1 0]
amount of columns left: 26


,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,...,open_acc,pub_rec,revol_bal,revol_util,total_acc,last_credit_pull_d,acc_now_delinq,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.0,Verified,1,credit_card,...,3.0,0.0,13648.0,83.7%,9.0,Jun-2016,0.0,0.0,0.0,0.0


In [34]:
## further cleaning of data: handling missing values ##

#import pandas as pd
loans=loans_2007

#check amount of missing values

null_counts=loans.isnull().sum()
print_null_counts=null_counts[null_counts!=0]
print('columns that contain NANs:')
print(print_null_counts)
#print(type(null_counts))

## Handling missing values: drop column with mostly NaNs, drop rows with mostly NaNs: ##

loans=loans.drop('pub_rec_bankruptcies',axis=1)
loans=loans.dropna(axis=0)
check=loans.isnull().sum()
#print(check[check>0])

print(' ')
print('value types in remaining data:')
print(loans.dtypes.value_counts())

columns that contain NANs:
emp_length              1036
title                     11
revol_util                50
last_credit_pull_d         2
pub_rec_bankruptcies     697
tax_liens                 39
dtype: int64
 
value types in remaining data:
float64    13
object     11
int64       1
dtype: int64


In [35]:
## turn categorical columns into numerical columns##

object_columns_df=loans.select_dtypes(include=['object'])
#print(object_columns_df.head(1))




## 7. Categorical columns ##

mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}

loans=loans.replace(mapping_dict)

## title and purpose give same information, purpose has less unique values: drop title; also drop other columns that can't be easliy converted into numbers ##

#print(loans['title'].value_counts())
#print(loans['purpose'].value_counts())
loans=loans.drop(['last_credit_pull_d', 'addr_state', 'title', 'earliest_cr_line'],axis=1)

#int_Rate and revol_util are numerical if % is removed:

for l in ['int_rate', 'revol_util']: 
    loans[l]=(loans[l].str.rstrip('%')).astype('float')

## Dummy variables for few unique categorical values ##
cols = ['home_ownership', 'verification_status', 'emp_length', 'term']
#for col in cols:
    #print(loans[col].value_counts())
    
dummy_df=pd.get_dummies(loans[['home_ownership','verification_status','purpose','term']])
loans=pd.concat([loans,dummy_df],axis=1)
loans=loans.drop(['home_ownership','verification_status','purpose','term'],axis=1)

loans.iloc[1,:]


loan_amnt                               2500
int_rate                               15.27
installment                            59.83
emp_length                                 0
annual_inc                             30000
loan_status                                0
dti                                        1
delinq_2yrs                                0
inq_last_6mths                             5
open_acc                                   3
pub_rec                                    0
revol_bal                               1687
revol_util                               9.4
total_acc                                  4
acc_now_delinq                             0
delinq_amnt                                0
tax_liens                                  0
home_ownership_MORTGAGE                    0
home_ownership_NONE                        0
home_ownership_OTHER                       0
home_ownership_OWN                         0
home_ownership_RENT                        1
verificati

## try different models : goal: conservative investor, get nearly all loans paid back
1 assume all loans are paid back <br>
2 Logistic Regression <br>
3 Random Forest Classifier

In [44]:


#print(loans.info())

## Picking an error metric ##


#tn=len(predictions[(predictions==0)&(loans['loan_status']==0)])
#tp=len(predictions[(predictions==1)&(loans['loan_status']==1)])
#fn=len(predictions[(predictions==0)&(loans['loan_status']==1)])
#fp=len(predictions[(predictions==1)&(loans['loan_status']==0)])



fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])
                   

#realize Class imbalance - will be dealt with later

# 1. first SIMPLE model: Realize good accuracy when predicting that all loans will be paid off on time (imbalance!)
predictions = pd.Series(numpy.ones(loans.shape[0]))

fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)
print('tpr and fpr of SIMPLE model: assume all loans are paid off:')
print(tpr, fpr)


## 2. Logistic Regression simple model ##

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
features=loans.drop('loan_status', axis=1).astype('float')
target=loans['loan_status'].astype('float')
lr.fit(features,target)
predictions=lr.predict(features)
#print(predictions.values_count())

fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

tpr=tp/(tp+fn)
fpr=fp/(tn+fp)
print('tpr and fpr of simple Logistic Regression:')
print(tpr, fpr)

## 2a. Logistic Regression including Cross Validation ##

#from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
lr = LogisticRegression()
predictions=cross_val_predict(lr,features,target,cv=3)
predictions=pd.Series(predictions)

fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

tpr=tp/(tp+fn)
fpr=fp/(tn+fp)
print('tpr and fpr of Logistic Regression with Cross Validation:')
print(tpr, fpr)

## 2b address the imbalance: Penalizing the classifier ##

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

lr = LogisticRegression(class_weight='balanced')
predictions=cross_val_predict(lr,features,target,cv=3)
predictions=pd.Series(predictions)

fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

tpr=tp/(tp+fn)
fpr=fp/(tn+fp)
print('tpr and fpr of Logistic Regression addressing imbalance and cross validation:')
print(tpr, fpr)

## 2c increase penalties manually##

#from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import cross_val_predict

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
penalty = {
    0: 10,
    1: 1
}

lr = LogisticRegression(class_weight=penalty)
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print('tpr and fpr of Logistic Regression with manual penalty:')
print(tpr, fpr)

## 3 Model Random forests ##

from sklearn.ensemble import RandomForestClassifier
#from sklearn.cross_validation import cross_val_predict
penalty={
    0:10,
    1:1
}


rf = RandomForestClassifier(class_weight='balanced',random_state=1)
predictions=cross_val_predict(rf,features,target,cv=3)
predictions=pd.Series(predictions)

fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

tpr=tp/(tp+fn)
fpr=fp/(tn+fp)
print('tpr and fpr of RandomForestClassifier:')
print(tpr, fpr)

tpr and fpr of SIMPLE model: assume all loans are paid off:
1.0 1.0


C:\Users\vikto00j\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


tpr and fpr of simple Logistic Regression:
0.9989766807243861 0.9970304380103935


C:\Users\vikto00j\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\vikto00j\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\vikto00j\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


tpr and fpr of Logistic Regression with Cross Validation:
0.998457954657305 0.9986163273374185


...

tpr and fpr of Logistic Regression addressing imbalance and cross validation:
0.6268578365431937 0.6127693220003954


C:\Users\vikto00j\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\vikto00j\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\vikto00j\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


tpr and fpr of Logistic Regression with manual penalty:
0.22930542340627974 0.22751531923305002


C:\Users\vikto00j\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\vikto00j\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\vikto00j\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


tpr and fpr of RandomForestClassifier:
0.9642376718396273 0.9626408381102984


## further tuning necessary
try ensemble of models, other models,...